In [14]:
from pytrends.request import TrendReq
import pandas as pd
import plotly.graph_objects as go

# Create a dictionary mapping state names to geo codes
state_geo_codes = {
    'Maharashtra': 'IN-MH',
    'Delhi': 'IN-DL',
    'Karnataka': 'IN-KA',
    'Gujarat': 'IN-GJ',
    'Tamil Nadu': 'IN-TN',
    'West Bengal': 'IN-WB',
    'Uttar Pradesh': 'IN-UP',
    'Rajasthan': 'IN-RJ',
    # Add more states as needed
}

# Prompt user to enter the state name
state_name = input("Enter the state name: ")

# Get the corresponding geo code from the dictionary
geo_code = state_geo_codes.get(state_name)

# Check if the entered state is valid
if geo_code is None:
    print(f"Invalid state name: {state_name}. Please enter a valid state from the list.")
else:
    # Initialize pytrends
    pytrends = TrendReq(hl='en-US', tz=360)

    # Prompt user to input specific men's wear items
    mens_wear_input = input("Enter the brands available  (comma-separated, e.g., Shirt, Jeans, Jacket): ")

    # Convert input string to a list of keywords
    mens_wear_kw = [item.strip() for item in mens_wear_input.split(',')]

    # Function to plot search interest for the selected category
    def plot_trends(kw_list, geo_code):
        # Build the payload for the selected state
        pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m', geo=geo_code, gprop='')

        # Get interest over time
        data = pytrends.interest_over_time()

        # Filter out the 'isPartial' column, if present
        if 'isPartial' in data.columns:
            data = data.drop(columns=['isPartial'])

        # Save the data to a dataframe
        df = pd.DataFrame(data)

        # Identify the top 2 most-searched items
        top_2 = df.sum().nlargest(2).index

        # Create an empty figure
        fig = go.Figure()

        # Add a line for each of the top 2 products
        for product in top_2:
            fig.add_trace(go.Scatter(x=df.index, y=df[product], mode='lines', name=product))

        # Update the layout with titles and labels
        fig.update_layout(
            title=f'Search Interest Over Time for Top 2 Searched provided Brands in  ({state_name})',
            xaxis_title='Date',
            yaxis_title='Search Interest',
            legend_title='Brands',
            xaxis_tickangle=-45
        )

        # Display the plot
        fig.show()

    # Plot the trends based on user input
    plot_trends(mens_wear_kw, geo_code)


In [15]:
def plot_pie_chart(kw_list, geo_code):
        # Build the payload for the selected state
        pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m', geo=geo_code, gprop='')

        # Get interest over time
        data = pytrends.interest_over_time()

        # Filter out the 'isPartial' column, if present
        if 'isPartial' in data.columns:
            data = data.drop(columns=['isPartial'])

        # Sum up the total interest for each brand over the time period
        total_interest = data.sum()

        # Create a pie chart using Plotly
        fig = go.Figure(data=[go.Pie(labels=total_interest.index, values=total_interest.values, hole=0.3)])

        # Update layout with title
        fig.update_layout(
            title=f'Search Interest Proportion for Entered Brands in ({state_name})',
        )

        # Show the pie chart
        fig.show()

In [16]:
plot_pie_chart(mens_wear_kw, geo_code)